In [1]:
using DataFrames

df = readtable("train.csv");

# Variablen-Typen
Für statistische Variablen lassen sich gewisse Kenngrössen bestimmen welche die 
**allgemeine/durchschnittliche Tendenz**, die **Streuung** oder die **Form** der zugrundeliegenden
Verteilung beschreiben. Je nach dem welche von diesen Grössen für den konkreten Fall (=Variable)
wohldefiniert sind, kann man zwischen 3 Haupttypen unterscheiden.

### Allgemeine Tendenz
- Arithmetisches Mittel: $\bar{x} = \frac{1}{n}\sum_{i=1}^n x_i$
- Median: Es gibt $\frac{n}{2}$ Werte die Grösser sind als der Median und $\frac{n}{2}$ die kleiner sind.
- Mode: Der am häufigsten vorkommende Wert.

### Streuung
- Gesamter Wertebereich (min bis max)
- Quantile (die ersten, zweiten, letzten.. x-ten Prozent)
- Varianz: $\sigma^2 = \frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2$
- Standardabweichung: $\sigma = \sqrt{\sigma^2}$

### Form
- Skewness (Schiefe): Wie symmetrisch ist die Verteilung? $\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^3$
- Kurtosis (Wölbung): Wie spitzig ist die Verteilung $\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^4$

# Quantitative Variablen
Lassen Berechnungen für alle oben genannten Kenngrössen zu und
werden in der Regel durch Reale Zahlen dargestellt, wie z.B:
- Körpergewicht in kg
- Temperatur in Grad Kelvin
- Kokainverbrauch pro Person pro Jahr
- Anzahl Einwohner

Weiter liegt Quantitativen Variablen eine sinnvolle numerische Skala zugrunde auf der sich eine
"Distanz" berechnen lässt. Als Beispiel ist es sinnvoll zu sagen 2000 kg sind 4 mal so viel wie 500 kg.

**Alle oben genannten Kenngrössen sind wohldefiniert.**

# Ordinale Variablen
Haben keine sinnvolle nuerische Skala als Grundlage, es gibt aber eine eindeutig definierte Reihenfolge der Elemente. Man kann also Elemente sortieren, aber keine sinnvolle euklidische Distanz zwischen ihen Bestimmen.
- Hausnummern
- T-shirt Grössen
- Akunto Abo-Preise

**Definierte Kenngrössen: Reihenfolge, Median, Mode, Quantile**

# Kategorische Variablen
Es lassen sich weder eine Reihenfolge, noch eine Distanz zwischen Elementen sinnvoll definieren.

- Farben
- Vornamen
- Nationalität

**Definierte Kenngrössen: Mode**






In [2]:
# Untersuche nochmals die Daten und Bestimme den Typ für jede Variable:
names(df)

12-element Array{Symbol,1}:
 :PassengerId
 :Survived   
 :Pclass     
 :Name       
 :Sex        
 :Age        
 :SibSp      
 :Parch      
 :Ticket     
 :Fare       
 :Cabin      
 :Embarked   